### Library

In [5]:
from datetime import datetime, timedelta
import oandapy
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sys
import os
import talib
sys.path.append('/Users/toshio/MyProject/fx')
from config import token

### Oandaのインスタンス化

In [6]:
access_token = token
oanda = oandapy.API(environment="practice", access_token=access_token)

### 過去データの取得

In [31]:
gran = 'H1'

In [10]:
i = 0
end = datetime(2018, 8, 1, 0, 0)
res = []
while i < 20:
    start = end - timedelta(hours = 4999)
    print(i, start, end)

    end_day =  '{0}-{1:02d}-{2:02d}-{3:02d}:{4:02d}:{5:02d}'.format(end.year, end.month, end.day, end.hour, end.minute, end.second)
    start_day =  '{0}-{1:02d}-{2:02d}-{3:02d}:{4:02d}:{5:02d}'.format(start.year, start.month, start.day, start.hour, start.minute, start.second)

    res.append(oanda.get_history(instrument="USD_JPY",granularity=gran, start = start_day, end = end_day))
    i += 1
    end = start - timedelta(hours = 1)

0 2018-01-04 17:00:00 2018-08-01 00:00:00
1 2017-06-10 09:00:00 2018-01-04 16:00:00
2 2016-11-14 01:00:00 2017-06-10 08:00:00
3 2016-04-19 17:00:00 2016-11-14 00:00:00
4 2015-09-24 09:00:00 2016-04-19 16:00:00
5 2015-02-28 01:00:00 2015-09-24 08:00:00
6 2014-08-03 17:00:00 2015-02-28 00:00:00
7 2014-01-07 09:00:00 2014-08-03 16:00:00
8 2013-06-13 01:00:00 2014-01-07 08:00:00
9 2012-11-16 17:00:00 2013-06-13 00:00:00
10 2012-04-22 09:00:00 2012-11-16 16:00:00
11 2011-09-27 01:00:00 2012-04-22 08:00:00
12 2011-03-02 17:00:00 2011-09-27 00:00:00
13 2010-08-06 09:00:00 2011-03-02 16:00:00
14 2010-01-10 01:00:00 2010-08-06 08:00:00
15 2009-06-15 17:00:00 2010-01-10 00:00:00
16 2008-11-19 09:00:00 2009-06-15 16:00:00
17 2008-04-25 01:00:00 2008-11-19 08:00:00
18 2007-09-29 17:00:00 2008-04-25 00:00:00
19 2007-03-05 09:00:00 2007-09-29 16:00:00


In [27]:
df = pd.DataFrame(res[0]['candles'])
for i in range(1, len(res)):
    df1 = pd.DataFrame(res[i]['candles'])
    df = pd.concat([df1, df], axis = 0)

In [28]:
df

,closeAsk,closeBid,complete,highAsk,highBid,lowAsk,lowBid,openAsk,openBid,time,volume
0,115.207,115.189,True,115.490,115.472,115.167,115.149,115.450,115.432,2007-03-05T09:00:00.000000Z,1380
1,115.306,115.288,True,115.427,115.409,115.216,115.198,115.218,115.200,2007-03-05T10:00:00.000000Z,1424
2,115.555,115.537,True,115.586,115.568,115.282,115.264,115.317,115.299,2007-03-05T11:00:00.000000Z,1365
3,115.575,115.557,True,115.705,115.687,115.435,115.417,115.558,115.540,2007-03-05T12:00:00.000000Z,1228
4,115.545,115.527,True,115.655,115.637,115.357,115.339,115.595,115.577,2007-03-05T13:00:00.000000Z,1593
5,115.935,115.905,True,116.001,115.983,115.484,115.466,115.535,115.517,2007-03-05T14:00:00.000000Z,1690
6,115.984,115.966,True,116.159,116.141,115.701,115.683,115.927,115.897,2007-03-05T15:00:00.000000Z,2084
7,116.024,116.006,True,116.256,116.238,115.971,115.953,115.974,115.956,2007-03-05T16:00:00.000000Z,1532
8,116.007,115.989,True,116.126,116.108,115.888,115.870,116.026,116.008,2007-03-05T17:00:00.000000Z,865
9,115.902,115.884,True,116.012,115.994,115.820,115.802,116.012,115.994,2007-03-05T18:00:00.000000Z,760


In [29]:
df = df.drop(['complete'], axis = 1)

df['time'] = df['time'].str[:-8]
df['time'] = df['time'].str.replace('T',' ')
times = [datetime.strptime(v, '%Y-%m-%d %H:%M:%S') for v in df['time']]
df['time'] = times
df = df.set_index('time',drop = True)

### 出力

In [33]:
with open('../intermediate_data/data_{}.pickle'.format(gran), mode='wb') as f:
    pickle.dump(df, f)